In [1]:
library(extrafont)
library(tidyverse)
library(epidemia)
library(arrow)

Registering fonts with R

── Attaching core tidyverse packages ───────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ─────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




# 1. parameters

In [2]:
T = 60
T0 = 40

## observation parameters

In [3]:
alpha = 0.01
r_Y = 10

$$Y_t = NegBinom( \frac{EY_t}{r_Y + EY_t}, r_Y )$$

$$ EY_t = \alpha\sum_{s=1}^{t-1} I_s \pi_{t-s} $$

## latent parameters

In [4]:
d = 2
beta = c(0.25, -2.45)

In [5]:
r_I = 100
K = 6.5

$$I_t = NegBinom( \frac{EI_t}{r_I + EI_t}, r_I )$$

$$ EI_t = R(\bar{A}_t, \beta) \sum_{s=1}^{t-1} I_s g_{t-s} $$

$$R(\bar{A}_t, \beta) = \frac{K}{1+\exp(- \beta^\top A_t)}$$

## seed values

In [6]:
mu = log(100)

$$I_{-T_0} = \mu_0$$

# 2. fit

In [7]:
num_sim = 1000

In [8]:
data <- read_feather('data/simulation_randinf_1.feather')

In [9]:
head(data)

sim,date,R,EI,infection,EY,death,intervention
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
0,0,3.654147,365.4122,399,0.9560691,0,0
0,1,3.654147,385.4360,503,0.9625439,0,0
0,2,3.654147,465.1591,437,0.9683041,0,0
0,3,3.654147,597.3970,649,0.9738399,1,0
0,4,3.654147,762.8805,808,0.9805697,1,0
0,5,3.654147,974.6201,782,0.9906314,1,0


In [10]:
rt <- epirt(formula = R(country, date) ~ 1 + intervention,
            prior = shifted_gamma(shape = 1/6, scale = 1, shift = log(1.05)/6),
            prior_covariance = decov(shape = c(2, rep(0.5, 5)), scale = 0.25),
            link = scaled_logit(6.5))

In [11]:
inf <- epiinf(gen = EuropeCovid$si, seed_days = T0)

In [12]:
deaths <- epiobs(formula = death ~ 1, i2o = EuropeCovid2$inf2death,
                 prior_intercept = normal(0, 0.2), link = scaled_logit(0.02))

Warning message:
“i2o does not sum to one. Please ensure this is intentional.”


In [13]:
results = array(0, c(num_sim, 6))
colnames(results)=c('b[0]','b[1]','V[0,0]','V[1,0]','V[0,1]','V[1,1]')

In [14]:
for(iter_sim in 1:num_sim){
    start_sim = proc.time()[3]
    
    data_raw = data %>% filter(sim == iter_sim-1)
    data_i = data.frame(country = factor(rep(1,T+T0)),
                        date = as.Date("2022-01-01")
                               + c(-T0:-1, data_raw$date),
                        death = c(rep(NA,T0), data_raw$death),
                        intervention = c(rep(0,T0), data_raw$intervention))
    
    args <- list(rt = rt, inf = inf, obs = deaths, data = data_i, 
                 seed=12345, refresh=0)
    args$algorithm <- "fullrank"; args$iter <- 5e4; args$tol_rel_obj <- 1e-3
    
    fm <- do.call(epim, args)
    
    results[iter_sim,1:2] = colMeans(as.data.frame(fm$stanfit)[1:2])
    results[iter_sim,3:6] = cov(as.data.frame(fm$stanfit)[1:2])
    
    print(paste0(iter_sim,"-th simulation finished, ",
                 proc.time()[3] - start_sim," sec."))
    flush(stdout())
}

[1] "1-th simulation finished, 5.53999999999996 sec."
[1] "2-th simulation finished, 5.48099999999977 sec."
[1] "3-th simulation finished, 5.26600000000008 sec."


Warning message:
“Pareto k diagnostic value is 0.71. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "4-th simulation finished, 4.66300000000001 sec."
[1] "5-th simulation finished, 5.31899999999996 sec."
[1] "6-th simulation finished, 4.79399999999987 sec."
[1] "7-th simulation finished, 5.3739999999998 sec."
[1] "8-th simulation finished, 4.76600000000008 sec."
[1] "9-th simulation finished, 4.75199999999995 sec."
[1] "10-th simulation finished, 5.32699999999977 sec."
[1] "11-th simulation finished, 5.19200000000001 sec."
[1] "12-th simulation finished, 5.23300000000017 sec."
[1] "13-th simulation finished, 5.29500000000007 sec."
[1] "14-th simulation finished, 5.27799999999979 sec."
[1] "15-th simulation finished, 3.375 sec."
[1] "16-th simulation finished, 5.33899999999994 sec."
[1] "17-th simulation finished, 4.721 sec."
[1] "18-th simulation finished, 5.24300000000039 sec."
[1] "19-th simulation finished, 5.28200000000015 sec."


Warning message:
“Pareto k diagnostic value is 0.72. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "20-th simulation finished, 5.24600000000009 sec."
[1] "21-th simulation finished, 2.17100000000028 sec."
[1] "22-th simulation finished, 5.23399999999992 sec."
[1] "23-th simulation finished, 5.20699999999988 sec."


...

[1] "935-th simulation finished, 4.57500000000073 sec."
[1] "936-th simulation finished, 5.11300000000119 sec."
[1] "937-th simulation finished, 4.59499999999935 sec."
[1] "938-th simulation finished, 5.07599999999911 sec."
[1] "939-th simulation finished, 5.16899999999987 sec."
[1] "940-th simulation finished, 4.61700000000019 sec."
[1] "941-th simulation finished, 4.63000000000102 sec."
[1] "942-th simulation finished, 5.09000000000015 sec."
[1] "943-th simulation finished, 5.09500000000116 sec."


Warning message:
“Pareto k diagnostic value is 0.74. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "944-th simulation finished, 4.54500000000007 sec."
[1] "945-th simulation finished, 5.04600000000028 sec."
[1] "946-th simulation finished, 5.14700000000084 sec."
[1] "947-th simulation finished, 5.05999999999949 sec."
[1] "948-th simulation finished, 5.1299999999992 sec."
[1] "949-th simulation finished, 4.63400000000001 sec."
[1] "950-th simulation finished, 5.13299999999981 sec."
[1] "951-th simulation finished, 5.10900000000038 sec."
[1] "952-th simulation finished, 5.09799999999996 sec."
[1] "953-th simulation finished, 5.12099999999919 sec."
[1] "954-th simulation finished, 5.07999999999993 sec."
[1] "955-th simulation finished, 5.15099999999984 sec."


Warning message:
“Pareto k diagnostic value is 0.73. Resampling is unreliable. Increasing the number of draws or decreasing tol_rel_obj may help.”


[1] "956-th simulation finished, 4.55099999999948 sec."
[1] "957-th simulation finished, 5.14300000000003 sec."
[1] "958-th simulation finished, 5.08400000000074 sec."
[1] "959-th simulation finished, 5.10299999999916 sec."
[1] "960-th simulation finished, 5.08500000000095 sec."
[1] "961-th simulation finished, 5.12800000000061 sec."
[1] "962-th simulation finished, 5.20100000000093 sec."
[1] "963-th simulation finished, 4.59599999999955 sec."
[1] "964-th simulation finished, 5.12900000000081 sec."
[1] "965-th simulation finished, 5.18000000000029 sec."
[1] "966-th simulation finished, 4.59799999999996 sec."
[1] "967-th simulation finished, 5.15200000000004 sec."
[1] "968-th simulation finished, 5.08500000000095 sec."
[1] "969-th simulation finished, 5.09900000000016 sec."
[1] "970-th simulation finished, 5.10799999999836 sec."
[1] "971-th simulation finished, 5.09600000000137 sec."
[1] "972-th simulation finished, 5.07200000000012 sec."
[1] "973-th simulation finished, 5.1149999999997

In [15]:
write_feather(as.data.frame(results), 
              'coverage_epidemia_nbinom_randinf_1.feather')